## Подготовка данных

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [43]:
data = pd.read_csv('../input/advanced-dls-spring-2021/train.csv')
#data = pd.read_csv('./train.csv') - если не на kaggle запускаете
print(data.shape, "- Размерность датасета")
data.head()

Посмотрим количество объектов типа NaN

In [44]:
data.isna().sum()

Видно, что объекты типа NaN отсутствуют

### Загрузка данных:

In [45]:
# Числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

feature_cols = num_cols + cat_cols
target_col = 'Churn'

In [46]:
X = data[feature_cols]
y = data[target_col]

### Анализ данных:

Для числовых признаков построим гистограммы распределения признаков:

При построении гистограммы для числового признака TotalSpent было замечено, что что-то не так, при исследовании ситуации я определил, что не все объекты там могли преобразоваться к типу float, поскольку значение в них было " ". Произведем отлов таких строчек и в зависимости от их количества будем решать, что с ними делать:

In [47]:
X.loc[X.TotalSpent == " " , 'TotalSpent'] = "0"
X = X.astype(
    {'ClientPeriod': float, 'MonthlySpending': float, 'TotalSpent': float}
)

Таких строчек вышло всего 9 штук, я решил заменить " " на нули в этих строчках

In [48]:
plt.figure(figsize=(20, 7))

sp = None
for num_col in num_cols:
    sp = plt.subplot(1, 3, num_cols.index(num_col) + 1)

    plt.hist(X[num_col])

    plt.ylabel("amount")
    plt.xlabel(num_col)
    plt.grid(alpha=0.2)

plt.show()

Видно, что числовые признаки хорошо распределены - они принимают множество различных значений, а не сосредоточены в одном месте

Для категориальных признаков посмотрим какое количество возможных значений имеет каждый признак:

In [49]:
for cat_col in cat_cols:
    print(
        f'{cat_col} : {X[cat_col].unique()}, различных : {len(X[cat_col].unique())}'
    )

Построим диаграммы распределения для категориальных признаков:

In [50]:
sp = None
plt.figure(figsize=(30, 20))
for cat_col in cat_cols:
    sp = plt.subplot(4, 4, cat_cols.index(cat_col) + 1)
    my_labels = X[cat_col].unique()
    my_values = X[cat_col].value_counts()
    plt.title(cat_col, fontdict={'fontsize': 20, 'color': 'w'})
    plt.pie(my_values, labels=my_labels, textprops={'color' : 'lightblue', 'fontsize' : 14})
plt.show()
None

Посмотрим на распределение целевой переменной:

In [51]:
plt.xlabel('Метка класса')
plt.ylabel('Число объектов')
plt.hist(y, bins=2)
print(
    len(y.loc[y == 0]), "- Количество объектов с меткой 0\n",
    len(y.loc[y == 1]), "- Количество объектов с меткой 1"
)
print(
    "Количество объектов 0 класса примерно в 3 раза превышает количество объектов класса 1 - можно считать, что выборка сбалансирована "
)

В качестве дополнительного пункта я решил посмотреть на вид матрицы линейной корелляции, по ее виду можно сразу понять, что линейная модель может показать довольно хороший результат

In [52]:
X[num_cols].corr()

Видно, что ClientPerid довольно хорошо коррелирует с TotalSpent, так что можно надеяться на неплохой результат для линейной модели

## Применение линейной модели

In [53]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer

Построим пайплайн из необходимых обработок данных и логистической регрессии, затем подберем оптимальные гиперпараметры через GridSearchCV и произведем оценку качества модели

In [54]:
X.TotalSpent

In [55]:
# Создаем отдельно обработчик данных, который будет стандартизировать числовые признаки и one-hot кодировать категориальные
col_transformer = make_column_transformer(
          (StandardScaler(), num_cols),
          (OneHotEncoder(), cat_cols)
          )

X_original = X.copy()

Чтобы была возможность как-то проверить качество модели до отправки на kaggle, я разделю обучающую выборку на train и valid, а уже датасет, результаты предсказания на котором я буду отправлять на kaggle, я потом назову X_submission

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=314)

In [57]:
X_train_transformed = col_transformer.fit_transform(X_train)
X_valid_transformed = col_transformer.transform(X_valid)

Произведем подбор лучших гиперпараметров при помощи кросс валидации

In [58]:
params = {
    'C': [10000, 1000, 100, 10, 1, 0.1, 0.01, 0.001],
    'penalty': ['l1', 'l2', 'elasticnet'],
}

search = GridSearchCV(
    LogisticRegression(solver='saga'),
    params,    
    cv = 5,
    scoring='roc_auc',
    n_jobs=-1
)

search.fit(X_train_transformed, y_train)
None

Оценим качество на X_valid , применив модель с наилучшими параметрами:

In [59]:
from sklearn.metrics import roc_auc_score

lin_estimator = search.best_estimator_

y_pred_train = lin_estimator.predict_proba(X_train_transformed)[:, 1]
y_pred_valid = lin_estimator.predict_proba(X_valid_transformed)[:, 1]
# predict_proba возвращает вероятности для всех классов, поэтому сейчас там 2 колонки, для roc-auc нам нужен 2 столбец

train_auc = roc_auc_score(y_train, y_pred_train)
valid_auc = roc_auc_score(y_valid, y_pred_valid)
print(train_auc, valid_auc)

Даже простая линейная модель уже показала 0.83 roc-auc скора на валидационной выборке, это довольно-таки хороший результат, если учитывать то, что результаты линейных моделей легко интерпретируемы. Теперь применим алгоритм бустинга, поскольку нас не волнует интерпретируемость, а волнует только точность.

## Применение градиентного бустинга

In [60]:
import catboost

In [61]:
boosting_model = catboost.CatBoostClassifier(n_estimators=300, 
                                             cat_features=cat_cols,
                                             l2_leaf_reg=1,
                                             depth=4
                                            )

boosting_model.fit(X_train, y_train)

y_train_predicted = boosting_model.predict_proba(X_train)[:, 1]
y_valid_predicted = boosting_model.predict_proba(X_valid)[:, 1]

In [62]:
train_auc = roc_auc_score(y_train, y_train_predicted)
valid_auc = roc_auc_score(y_valid, y_valid_predicted)

print(train_auc, valid_auc)

## Kaggle

Сделаем файл, который будем отправлять на kaggle, для этого надо подготовить датасет, на котором будем делать предсказания для отправки

In [63]:
X_submission = pd.read_csv('../input/advanced-dls-spring-2021/test.csv')
#X_submission = pd.read_csv('./test.csv') - если не на kaggle запускаете

X_submission.loc[X_submission.TotalSpent == " " , 'TotalSpent'] = "0"
X_submission = X_submission.astype(
    {'ClientPeriod': float, 'MonthlySpending': float, 'TotalSpent': float}
)

Зафитим и в ту и в другую модели полный X, чтобы улучшить качество предсказания на X_submission. Я попробую отправить на kaggle результаты обеих моделей, потому что их результаты на валидационной выборке очень близки

In [64]:
X_transformed = col_transformer.fit_transform(X)
X_submission_transformed = col_transformer.transform(X_submission)

search.fit(X_transformed, y)

lin_estimator = search.best_estimator_

In [65]:
linear_model_prediction_for_submission = lin_estimator.predict_proba(X_submission_transformed)[:, 1]

Теперь зафитим весь X в бустинг:

In [66]:
boosting_model.fit(X, y)

In [67]:
boosting_prediction_for_submission = boosting_model.predict_proba(X_submission)[:, 1]

In [68]:
linear_model_prediction_for_submission

In [69]:
boosting_prediction_for_submission

In [70]:
submission_lin = pd.read_csv('../input/advanced-dls-spring-2021/submission.csv', index_col='Id')
submission_boost = pd.read_csv('../input/advanced-dls-spring-2021/submission.csv', index_col='Id')
#submission_lin = pd.read_csv('./submission.csv', index_col='Id') - если не на kaggle
#submission_boost = pd.read_csv('./submission.csv', index_col='Id') - если не на kaggle запускаете

submission_lin['Churn'] = linear_model_prediction_for_submission
submission_boost['Churn'] = boosting_prediction_for_submission

In [71]:
submission_lin.to_csv('/kaggle/working/my_linear_submission.csv')
submission_boost.to_csv('/kaggle/working/my_boosting_submission.csv')

#submission_lin.to_csv('./my_linear_submission.csv') - если не на kaggle запускаете
#submission_boost.to_csv('./my_boosting_submission.csv') - если не на kaggle запускаете

### На kaggle в итоге я получил точность 0.85 с чем-то на бустинге и 0.84 с чем-то на линейной модели, ищите меня в табличке соревнования на kaggle - alphonse16